In [ ]:
!pip install transformers datasets -q

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, TFAutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('data_bersih_klasifikasi - data_bersih_klasifikasi.csv')
df = df[['cleaned_nostop', 'fine_tuned_label']]  # pastikan kolom benar
df.dropna(inplace=True)

In [5]:
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['fine_tuned_label'])
labels = to_categorical(df['label_encoded'], num_classes=3)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
bert_model = TFAutoModel.from_pretrained("indobenchmark/indobert-base-p1")

encoded = tokenizer(
    df['cleaned_nostop'].tolist(),
    padding=True,
    truncation=True,
    max_length=64,
    return_tensors='tf'
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to

In [1]:
bert_output = bert_model(encoded)
embeddings = bert_output.last_hidden_state[:, 0, :]  # Ambil token [CLS]

NameError: name 'bert_model' is not defined

In [2]:
X_train, X_test, y_train, y_test = train_test_split(
    embeddings.numpy(), labels, test_size=0.2, stratify=labels, random_state=42
)

NameError: name 'train_test_split' is not defined

In [3]:
model = Sequential()
model.add(tf.keras.Input(shape=(768,)))
model.add(tf.keras.layers.Reshape((1, 768)))
model.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

NameError: name 'Sequential' is not defined

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
    ]
)

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

# Grafik val_loss dan val_accuracy
plt.plot(history.history['val_loss'], label='Val Loss')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title("Validation Loss & Accuracy")
plt.show()

In [ ]:
def predict_sentiment(text):
    tokens = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=64)
    output = bert_model(tokens)
    cls_embedding = output.last_hidden_state[:, 0, :].numpy()
    prediction = model.predict(cls_embedding)
    label_index = np.argmax(prediction)
    label = label_encoder.inverse_transform([label_index])[0]
    return label

# Contoh
print(predict_sentiment("Video ini sangat bermanfaat!"))
print(predict_sentiment("Saya tidak suka sama isi kontennya"))